## 0. 환경 구성

### 1) 라이브러리 설치

In [ ]:
# poetry add langchain_community faiss-cpu

### 2) OpenAI 인증키 설정
https://openai.com/

In [2]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_t


In [3]:
EMBEDDING_MODEL_NAME = "bge-m3:latest"

#### RAG 파이프 라인
* Load Data - Text Split - Indexing - Retrieval - Generation
* OllamaEmbeddings 사용

In [6]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings

from pprint import pprint

# 1. Load Data
loader = TextLoader("../data/taxinfo.txt", encoding="utf-8")
documents = loader.load()
print(type(documents), len(documents))
print(type(documents[0]))
#print(documents[0])

<class 'list'> 1
<class 'langchain_core.documents.base.Document'>


In [8]:
# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)

print(len(split_docs), type(split_docs))
print(split_docs)

12 <class 'list'>
[Document(metadata={'source': '../data/taxinfo.txt'}, page_content='제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득'), Document(metadata={'source': '../data/taxinfo.txt'}, page_content='가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n    다. 대통령령으로 정하는 농어가부업소득\n    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이

In [9]:
# 3️. Indexing (벡터 저장)
embeddings_model = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
vectorstore = FAISS.from_documents(
    documents=split_docs, 
    embedding=embeddings_model
)
#vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

In [10]:
# 4️. Retrieval (유사 문서 검색) k: 질의와 가장 유사한 문서(청크) 6개를 찾아 반환하기
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
print(type(retriever))
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print(retrieved_docs)

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
[Document(id='fe0dab2a-f1b9-463a-b38d-4944ca3f4ad5', metadata={'source': '../data/taxinfo.txt'}, page_content='제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득'), Document(id='3640a87b-d80f-4bd6-9917-e6624cd9a299', metadata={'source': '../data/taxinfo.txt'}, page_content='가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다

In [11]:
# 5️. Generation (LLM 응답 생성)
#llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(context)

제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31.>
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득
    가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득

가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득
    나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.
    다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.
    바. 대통령령으로 정하는 작물재배업에서 발생하는 소득

바. 대통령령으로 정하는 작물재배업에서 발생하는 소득
    사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득
3. 근로소득과

In [12]:
response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용한 결과
('소득세법 제12조(비과세소득)에 따라 소득세를 **전혀 부과하지 않는 소득**은 다음과 같이 정리할 수 있습니다.  \n'
 '(※ 아래는 2023. 12. 31. 개정 기준으로, 실무 적용 시에는 시행령·시행규칙, 조세특례제한법, 유권해석 등을 함께 확인해야 '
 '합니다.)\n'
 '\n'
 '---\n'
 '\n'
 '### 1. 공익신탁 이익\n'
 '- 「공익신탁법」에 따른 공익신탁으로부터 수익자(공익법인 등)가 받는 이익\n'
 '\n'
 '---\n'
 '\n'
 '### 2. 사업소득 중 비과세\n'
 '가. **농지(논·밭)를 작물 생산에 이용하게 함으로써 발생하는 소득**  \n'
 '나. **1주택 1세대**가 얻는 주택임대소득(시가 12억 초과·국외주택 제외)  \n'
 '  - 2018. 12. 31.까지 끝나는 과세기간 한정, 연 2천만원 이하  \n'
 '다. **대통령령이 정하는 농어가부업소득**  \n'
 '라. **전통주 제조소득**  \n'
 '마. **조림 5년 이상 임지의 임목 벌채·양도소득** → 연 600만원 이하분  \n'
 '바. **대통령령이 정하는 작물재배업 소득**  \n'
 '사. **대통령령이 정하는 어로·양식어업 소득**\n'
 '\n'
 '---\n'
 '\n'
 '### 3. 근로·퇴직소득 중 비과세\n'
 '가. **현역 병사 급여**  \n'
 '나. **법령에 의한 동원근로자 급여**  \n'
 '다. **산재보험 급여**(요양·휴업·장해·간병·유족·장의비 등)  \n'
 '라. **근기법·선원법상 보상금**(요양·휴업·상병·장해·유족·장의비 등)  \n'
 '마. **「국군포로법」 연금**  \n'
 '바. **「산업재해보상보험법」상 각종 연금**  \n'
 '사. **종교인소득** 중  \n'
 '  - 학자금, 식사(대), 실비변상성 지급액  \n'
 '  - 출산·보육지원금(월 20만원 한도)  \n'
 '  - 사택 제공 이익  \n'
 '아. **보수를 받

### 개선한 Source - version1
* Retriever 검색방법 개선
    * search_type="mmr",  # 최대 다양성 검색
    * search_kwargs={"k": 6, "fetch_k": 10}  

In [13]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings

from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("../data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
embeddings_model = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

vectorstore = FAISS.from_documents(
    documents=split_docs, 
    embedding=embeddings_model
)
#vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())

vectorstore.save_local("faiss_index")

# 4. 검색 개선
"""
    최대 다양성 검색(Maximum Marginal Relevance, MMR)
    MMR은 유사도가 높은 문서를 찾는 것을 넘어, 유사도와 다양성이라는 두 가지 기준을 모두 고려험
    - search_type="mmr": 검색 방식을 MMR로 지정합니다.
    - fetch_k: 일차적으로 질의와 유사한 문서 10개를 벡터 저장소에서 가져옵니다.
    - k: fetch_k로 가져온 10개의 문서 중에서 최종적으로 6개를 선택합니다. 6개를 선택할 때, MMR 알고리즘은 다음 두 가지를 고려함
        : 질의와의 유사도가 높고, 이미 선택된 다른 문서들과의 유사도가 낮은 (즉, 내용이 다양한) 문서
    * MMR의 작동 원리:
    - 질의와 가장 유사한 fetch_k개(10개)의 문서를 예비 후보군으로 가져옵니다.
    - 이 후보군 중에서 질의와 가장 유사한 문서 하나를 첫 번째 결과로 선택합니다.
    - 남은 후보군 중에서 질의와의 유사도는 높으면서 (관련성), 
      이미 선택된 문서들과의 유사도는 낮은 (다양성) 문서를 찾아 다음 결과로 추가함
    - 이 과정을 k개(6개)의 문서가 모두 선택될 때까지 반복합니다.        
"""
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 6, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)  # 창의성 낮춤
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

개선된 결과:
('소득세법 제12조 비과세소득은 5대 유형으로 계층화된다.\n'
 '\n'
 '1. 공익·보훈·국가보안(5가·나·다·마·바·사·아·자목)  \n'
 '   - 공익신탁 이익, 보훈·북한이탈주민 급여, 국가보안 상금, 국군포로·문화재 양도·종교인 실비·보육비(월 20만원 한도) 등\n'
 '\n'
 '2. 농어업·임업·전통주(2가·나·다·라·마·바·사목)  \n'
 '   - 논·밭 임대, 1주택·연 2천만원 이하 임대(2018과세기간까지), 농어가부업, 전통주 제조, 5년 이상 조림지 벌채(연 '
 '600만원 이하), 대통령령 작물·어로·양식업 소득\n'
 '\n'
 '3. 근로·퇴직 급여(3가~어목)  \n'
 '   - 병역급여, 산재·고용보험급여, 육아·출산수당(월 20만원 이하), 장학금(대학생 근로대가), 직무발명보상금(대통령령 한도), '
 '식사대(월 20만원 이하) 등\n'
 '\n'
 '4. 공적연금·유족·장애연금(4가·다·라목)  \n'
 '   - 국민·공무원·군인·사립학교·별정우체국법 연계연금, 산재연금, 국군포로연금\n'
 '\n'
 '5. 기타(5가·나·다·라·마·바·사·아·자목)  \n'
 '   - 훈장 부상, 퇴직 후 직무발명보상금, 문화재 양도·기증 소득, 위원회 수당 등')


### 개선한 Source - version2
* Prompt 개선

In [14]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings

from pprint import pprint

# 1. Load Data
loader = TextLoader("../data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
embeddings_model = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

vectorstore = FAISS.from_documents(
    documents=split_docs, 
    embedding=embeddings_model
)

#vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

=== 원본 문서 길이 ===
전체 문서 길이: 4971 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.
    바. 대통령령으로 정하는 작물재배업에서 발생하...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용

In [15]:
# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)


=== 검색 방식 개선 테스트 ===
=== MMR 검색 결과 ===
('소득세법 제12조(비과세소득)에 따라 소득세를 부과하지 아니하는 소득은 다음과 같이 대분류·중분류·소분류로 정리할 수 있습니다.  \n'
 '(※ 아래 “대통령령으로 정하는 …”은 모두 「소득세법 시행령」 제19조~제19의13조에 위임된 사항임)\n'
 '\n'
 '--------------------------------------------------\n'
 '1. 사업소득\n'
 '1-1 농림·어업 관련  \n'
 '- 논·밭을 작물 생산에 이용한 임대소득  \n'
 '- 농어가부업소득(시행령 §19의2)  \n'
 '- 전통주 제조소득(시행령 §19의3)  \n'
 '- 조림 5년 이상 임지의 벌채·양도소득(연 600만원 한도)  \n'
 '- 시행령이 정하는 작물재배·어로·양식어업 소득  \n'
 '\n'
 '1-2 주택임대소득  \n'
 '- 1주택(기준시가 12억 이하, 국내소재) 임대소득  \n'
 '- 과세기간 총수입 2천만원 이하 주택임대소득(2018.12.31.까지 끝난 과세기간 한정)\n'
 '\n'
 '--------------------------------------------------\n'
 '2. 근로·퇴직소득\n'
 '2-1 군·동원·보상성 급여  \n'
 '- 복무 중 병사 급여  \n'
 '- 법률에 따른 동원근로 급여  \n'
 '- 산재보험 요양·휴업·장해·간병·유족급여 등  \n'
 '- 근기법·선원법상 각종 보상금(요양·휴업·장해·유족·장의비 등)  \n'
 '\n'
 '2-2 고용·공무원·연금 관련  \n'
 '- 실업급여, 육아휴직·출산전후·근로시간단축 급여  \n'
 '- 공무원·사립학교·별정우체국 육아휴직수당(일정한도 이하)  \n'
 '- 공익신탁 이익\n'
 '\n'
 '--------------------------------------------------\n'
 '3. 연금소득\n'
 '- 공적연금(국민·공무원·군인·사학·별정